# Cleaning Processes

In [351]:
# Imports
import pandas as pd
import re

In [352]:
# Load the data
citybikes = pd.read_csv('./sql_ready/citybikes.csv')
fsq = pd.read_csv('./raw/foursquare.csv')
yelp = pd.read_csv('./raw/yelp.csv')

print(f"{citybikes.shape} -- Citybikes is here.")
print(f"{fsq.shape} -- FSQ is here.")
print(f"{yelp.shape} -- Yelp is here.")

(190, 7) -- Citybikes is here.
(8698, 12) -- FSQ is here.
(10022, 12) -- Yelp is here.


### Cleaning Addresses

In [353]:
# FSQ
fa = fsq.copy()
fa['poi_address'][0]

'328 James St N (Barton Street West), Hamilton ON L8L 1H2'

In [354]:
# Remove "Hamilton, ON"
fa['poi_address'] = fa['poi_address'].str.replace('Hamilton ON', '')

# Separate postal code
fa['postal_code'] = fa['poi_address'].str.extract(r'(\w\d\w\s\d\w\d)')

# Remove postal code
fa['poi_address'] = fa['poi_address'].str.replace(r'(\w\d\w\s\d\w\d)', '', regex=True)

# Remove comma
fa['poi_address'] = fa['poi_address'].str.replace(',', '')

# Remove leading and trailing spaces
fa['poi_address'] = fa['poi_address'].str.strip()

In [355]:
fa['poi_address'][0]

'328 James St N (Barton Street West)'

In [356]:
fa.head(1)

,fsq_id,poi_name,poi_category_id,poi_rating,poi_review_count,poi_latitude,poi_longitude,poi_address,postal_code,station_id,distance_from_station,station_name
0,5ba0f82a15173e002c280c07,Synonym,13065,NaN,NaN,43.264558,-79.865376,328 James St N (Barton Street West),L8L 1H2,024a3edf037cb411d16acc08a7fcb954,420,Bay at Strachan


In [357]:
# Save
fa.to_csv('./raw/semi_processed/fa_foursquare.csv', index=False)

In [358]:
# Yelp
ya = yelp.copy()
ya['poi_address'][0]

"['200 Harbour Front Drive', 'Hamilton, ON L8L 1C8', 'Canada']"

In [359]:
# Remove brackets and quotes
ya['poi_address'] = ya['poi_address'].str.replace('[', '')
ya['poi_address'] = ya['poi_address'].str.replace(']', '')
ya['poi_address'] = ya['poi_address'].str.replace("'", '')

# Remove "Hamilton, ON" and "Canada"
ya['poi_address'] = ya['poi_address'].str.replace('Hamilton, ON', '')
ya['poi_address'] = ya['poi_address'].str.replace(' Canada', '')

# Remove comma(s)
ya['poi_address'] = ya['poi_address'].str.replace(',', '')

# Separate postal code
ya['postal_code'] = ya['poi_address'].str.extract(r'(\w\d\w\s\d\w\d)')

# Remove postal code
ya['poi_address'] = ya['poi_address'].str.replace(r'(\w\d\w\s\d\w\d)', '', regex=True)

# Remove extra spaces
ya['poi_address'] = ya['poi_address'].str.strip()

In [360]:
ya['poi_address'][0]

'200 Harbour Front Drive'

In [361]:
ya.head(1)

,yelp_id,poi_name,poi_category_id,poi_rating,poi_review_count,poi_latitude,poi_longitude,poi_address,postal_code,station_id,distance_from_station,station_name
0,fNfZ0bOEvXOxMP1vDEDi8w,Pirate Life Theatre,amusementparks,0.0,0,43.268525,-79.870901,200 Harbour Front Drive,L8L 1C8,024a3edf037cb411d16acc08a7fcb954,242.169774,Bay at Strachan


In [362]:
# Save
ya.to_csv('./raw/semi_processed/ya_yelp.csv', index=False)

### Cleaning to obtain POI data only.

#### Foursquare

In [363]:
f_clean = fsq.copy()

# Inspect
print(f_clean.shape)
print(f_clean.columns)

(8698, 12)
Index(['fsq_id', 'poi_name', 'poi_category_id', 'poi_rating',
       'poi_review_count', 'poi_latitude', 'poi_longitude', 'poi_address',
       'postal_code', 'station_id', 'distance_from_station', 'station_name'],
      dtype='object')


In [364]:
# Drop station data...
f_clean.drop(columns=['station_id', 'distance_from_station', 'station_name'], inplace=True)

print(f_clean.columns)

Index(['fsq_id', 'poi_name', 'poi_category_id', 'poi_rating',
       'poi_review_count', 'poi_latitude', 'poi_longitude', 'poi_address',
       'postal_code'],
      dtype='object')


In [365]:
# Sort by poi_name and poi_address
f_clean.sort_values(by=['poi_name', 'poi_address'], inplace=True)

# Call duplicates
f_clean[f_clean.duplicated(subset=['poi_name', 'poi_address'])]

,fsq_id,poi_name,poi_category_id,poi_rating,poi_review_count,poi_latitude,poi_longitude,poi_address,postal_code
1023,5dd2e5df0cd86e00081bd28b,1101 Convenience,13065,NaN,NaN,43.248628,-79.821565,"1101 Cannon St E (Kensington), Hamilton ON L8L...",NaN
1125,5dd2e5df0cd86e00081bd28b,1101 Convenience,13065,NaN,NaN,43.248628,-79.821565,"1101 Cannon St E (Kensington), Hamilton ON L8L...",NaN
4111,5dd2e5df0cd86e00081bd28b,1101 Convenience,13065,NaN,NaN,43.248628,-79.821565,"1101 Cannon St E (Kensington), Hamilton ON L8L...",NaN
5252,5dd2e5df0cd86e00081bd28b,1101 Convenience,13065,NaN,NaN,43.248628,-79.821565,"1101 Cannon St E (Kensington), Hamilton ON L8L...",NaN
6274,5dd2e5df0cd86e00081bd28b,1101 Convenience,13065,NaN,NaN,43.248628,-79.821565,"1101 Cannon St E (Kensington), Hamilton ON L8L...",NaN
...,...,...,...,...,...,...,...,...,...
963,5a61374d23a2e61ce49b0922,Zyla's Music and Menu,13065,NaN,NaN,43.264002,-79.866070,"299 James St N, Hamilton ON L8R 2L4",NaN
2202,5a61374d23a2e61ce49b0922,Zyla's Music and Menu,13065,NaN,NaN,43.264002,-79.866070,"299 James St N, Hamilton ON L8R 2L4",NaN
4402,5a61374d23a2e61ce49b0922,Zyla's Music and Menu,13065,NaN,NaN,43.264002,-79.866070,"299 James St N, Hamilton ON L8R 2L4",NaN
7034,5a61374d23a2e61ce49b0922,Zyla's Music and Menu,13065,NaN,NaN,43.264002,-79.866070,"299 James St N, Hamilton ON L8R 2L4",NaN


In [366]:
# Address cleaning
f_clean['poi_address'][0]

'328 James St N (Barton Street West), Hamilton ON L8L 1H2'

In [367]:
# Remove "Hamilton, ON"
f_clean['poi_address'] = f_clean['poi_address'].str.replace('Hamilton ON', '')

# Separate postal code
f_clean['postal_code'] = f_clean['poi_address'].str.extract(r'(\w\d\w\s\d\w\d)')

# Remove postal code
f_clean['poi_address'] = f_clean['poi_address'].str.replace(r'(\w\d\w\s\d\w\d)', '', regex=True)

# Remove comma
f_clean['poi_address'] = f_clean['poi_address'].str.replace(',', '')

# Remove leading and trailing spaces
f_clean['poi_address'] = f_clean['poi_address'].str.strip()

In [368]:
f_clean['poi_address'][0]

'328 James St N (Barton Street West)'

In [369]:
fsq_new = f_clean.drop_duplicates(subset=['poi_name', 'poi_address'], keep='first')

In [370]:
print(fsq_new.shape)
print(fsq_new.nunique())

(666, 9)
fsq_id              666
poi_name            586
poi_category_id       3
poi_rating            0
poi_review_count      0
poi_latitude        598
poi_longitude       605
poi_address         586
postal_code         346
dtype: int64


In [371]:
# Save a copy...
fsq_new.to_csv('./pois_only/pois_foursquare.csv', index=False)

#### Yelp

In [372]:
y_clean = yelp.copy()

# Inspect
print(y_clean.shape)
print(y_clean.columns)

(10022, 12)
Index(['yelp_id', 'poi_name', 'poi_category_id', 'poi_rating',
       'poi_review_count', 'poi_latitude', 'poi_longitude', 'poi_address',
       'postal_code', 'station_id', 'distance_from_station', 'station_name'],
      dtype='object')


In [373]:
# Drop station data...
y_clean.drop(columns=['station_id', 'distance_from_station', 'station_name'], inplace=True)

print(y_clean.columns)

Index(['yelp_id', 'poi_name', 'poi_category_id', 'poi_rating',
       'poi_review_count', 'poi_latitude', 'poi_longitude', 'poi_address',
       'postal_code'],
      dtype='object')


In [374]:
# Sort by poi_name and poi_address
y_clean.sort_values(by=['poi_name', 'poi_address'], inplace=True)

# Call duplicates
y_clean[y_clean.duplicated(subset=['poi_name', 'poi_address'])]

,yelp_id,poi_name,poi_category_id,poi_rating,poi_review_count,poi_latitude,poi_longitude,poi_address,postal_code
1043,CODMFMsPP06eTQutMnniUA,541 Eatery & Exchange,restaurants,4.5,45,43.258248,-79.842354,"['541 Barton Street E', 'Hamilton, ON L8L 2Z2'...",L8L 2Z2
1144,CODMFMsPP06eTQutMnniUA,541 Eatery & Exchange,restaurants,4.5,45,43.258248,-79.842354,"['541 Barton Street E', 'Hamilton, ON L8L 2Z2'...",L8L 2Z2
1261,CODMFMsPP06eTQutMnniUA,541 Eatery & Exchange,restaurants,4.5,45,43.258248,-79.842354,"['541 Barton Street E', 'Hamilton, ON L8L 2Z2'...",L8L 2Z2
1316,CODMFMsPP06eTQutMnniUA,541 Eatery & Exchange,restaurants,4.5,45,43.258248,-79.842354,"['541 Barton Street E', 'Hamilton, ON L8L 2Z2'...",L8L 2Z2
1427,CODMFMsPP06eTQutMnniUA,541 Eatery & Exchange,restaurants,4.5,45,43.258248,-79.842354,"['541 Barton Street E', 'Hamilton, ON L8L 2Z2'...",L8L 2Z2
...,...,...,...,...,...,...,...,...,...
3223,gI8lk0ROJVTR4f8BUVZClQ,sams tavern,restaurants,0.0,0,43.256802,-79.838188,"['632 Barton Street E', 'Hamilton, ON L8L 2Z9'...",L8L 2Z9
3542,gI8lk0ROJVTR4f8BUVZClQ,sams tavern,restaurants,0.0,0,43.256802,-79.838188,"['632 Barton Street E', 'Hamilton, ON L8L 2Z9'...",L8L 2Z9
3805,gI8lk0ROJVTR4f8BUVZClQ,sams tavern,restaurants,0.0,0,43.256802,-79.838188,"['632 Barton Street E', 'Hamilton, ON L8L 2Z9'...",L8L 2Z9
5334,gI8lk0ROJVTR4f8BUVZClQ,sams tavern,restaurants,0.0,0,43.256802,-79.838188,"['632 Barton Street E', 'Hamilton, ON L8L 2Z9'...",L8L 2Z9


In [375]:
# Address cleaning
y_clean['poi_address'][0]

"['200 Harbour Front Drive', 'Hamilton, ON L8L 1C8', 'Canada']"

In [376]:
# Remove brackets and quotes
y_clean['poi_address'] = y_clean['poi_address'].str.replace('[', '')
y_clean['poi_address'] = y_clean['poi_address'].str.replace(']', '')
y_clean['poi_address'] = y_clean['poi_address'].str.replace("'", '')

# Remove "Hamilton, ON" and "Canada"
y_clean['poi_address'] = y_clean['poi_address'].str.replace('Hamilton, ON', '')
y_clean['poi_address'] = y_clean['poi_address'].str.replace(' Canada', '')

# Remove comma(s)
y_clean['poi_address'] = y_clean['poi_address'].str.replace(',', '')

# Separate postal code
y_clean['postal_code'] = y_clean['poi_address'].str.extract(r'(\w\d\w\s\d\w\d)')

# Remove postal code
y_clean['poi_address'] = y_clean['poi_address'].str.replace(r'(\w\d\w\s\d\w\d)', '', regex=True)

# Remove extra spaces
y_clean['poi_address'] = y_clean['poi_address'].str.strip()

In [377]:
y_clean['poi_address'][0]

'200 Harbour Front Drive'

In [378]:
yp_new = y_clean.drop_duplicates(subset=['poi_name', 'poi_address'], keep='first')

In [379]:
print(yp_new.shape)
print(yp_new.nunique())

(528, 9)
yelp_id             528
poi_name            485
poi_category_id       3
poi_rating           41
poi_review_count     74
poi_latitude        493
poi_longitude       500
poi_address         485
postal_code         323
dtype: int64


In [380]:
# Save a copy...
yp_new.to_csv('./pois_only/pois_yelp.csv', index=False)

### Combining POI data from Foursquare and Yelp

In [381]:
# Import CSVs from the previous step
fsq_c = pd.read_csv('./pois_only/pois_foursquare.csv')
yelp_c = pd.read_csv('./pois_only/pois_yelp.csv')

print(f"{fsq_c.shape} -- FSQ is here.") # Matched the previous step
print(f"{yelp_c.shape} -- Yelp is here.") # Matched the previous step

(666, 9) -- FSQ is here.
(528, 9) -- Yelp is here.


In [382]:
# Columns
print(fsq_c.columns)
print(yelp_c.columns)

Index(['fsq_id', 'poi_name', 'poi_category_id', 'poi_rating',
       'poi_review_count', 'poi_latitude', 'poi_longitude', 'poi_address',
       'postal_code'],
      dtype='object')
Index(['yelp_id', 'poi_name', 'poi_category_id', 'poi_rating',
       'poi_review_count', 'poi_latitude', 'poi_longitude', 'poi_address',
       'postal_code'],
      dtype='object')


In [383]:
# I want to rename the poi_category_id columns to match the source, so I can fill in the missing values later
fsq_c.rename(columns={'poi_category_id' : 'f_category_id'}, inplace=True)
yelp_c.rename(columns={'poi_category_id' : 'y_category_id'}, inplace=True)

In [384]:
# Merge the two dataframes
pois_only = pd.concat([fsq_c, yelp_c], axis=0)

print(pois_only.shape)
print(pois_only.nunique())

(1194, 11)
fsq_id               666
poi_name             932
f_category_id          3
poi_rating            41
poi_review_count      74
poi_latitude        1083
poi_longitude       1103
poi_address         1047
postal_code          465
yelp_id              528
y_category_id          3
dtype: int64


In [385]:
pois_only.columns

Index(['fsq_id', 'poi_name', 'f_category_id', 'poi_rating', 'poi_review_count',
       'poi_latitude', 'poi_longitude', 'poi_address', 'postal_code',
       'yelp_id', 'y_category_id'],
      dtype='object')

In [386]:
print(pois_only.shape)
print(pois_only.nunique())

(1194, 11)
fsq_id               666
poi_name             932
f_category_id          3
poi_rating            41
poi_review_count      74
poi_latitude        1083
poi_longitude       1103
poi_address         1047
postal_code          465
yelp_id              528
y_category_id          3
dtype: int64


In [387]:
# Call duplicates, sort by poi_name
pois_only[pois_only.duplicated(subset=['poi_name'])].sort_values(by='poi_name')

,fsq_id,poi_name,f_category_id,poi_rating,poi_review_count,poi_latitude,poi_longitude,poi_address,postal_code,yelp_id,y_category_id
3,NaN,541 Eatery & Exchange,NaN,4.5,45.0,43.258248,-79.842354,541 Barton Street E,L8L 2Z2,CODMFMsPP06eTQutMnniUA,restaurants
8,NaN,A&W,NaN,2.0,1.0,43.250817,-79.807805,Centre Mall 1275 Barton Street E,L8H 2V4,I2AnCv6ny7_kC7MAT2clGQ,restaurants
7,NaN,A&W,NaN,2.8,4.0,43.262025,-79.857941,190 Barton Street E,L8L 2W8,GVdfb5969k4hQcnVMGKD6w,restaurants
9,503544ede4b0cde75df3b149,A&W Restaurant,13065.0,NaN,NaN,43.262054,-79.857951,190 Barton St E,L8L 2W8,NaN,NaN
10,4fe53cd6e4b0117428807583,A&W Restaurant,13065.0,NaN,NaN,43.258550,-79.875118,194 King St W,L8P 1A5,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
514,NaN,Williams Fresh Cafe,NaN,2.7,22.0,43.257587,-79.917962,1309 Main Street W Unit A-2,L8S 1C5,ZnDcWEpGFTwMqmXhpCa6fQ,restaurants
515,NaN,Williams Fresh Cafe,NaN,2.7,43.0,43.276487,-79.860838,47 Discovery Drive,L8L 8K4,iIpLGj-seyOgfOEtJMZlhQ,restaurants
650,4cc096adaa5776b0366e7544,Williams Fresh Cafe,13065.0,NaN,NaN,43.276538,-79.860908,47 Discovery Dr (Guise St. E.),L8L 8B4,NaN,NaN
513,NaN,Williams Fresh Cafe,NaN,5.0,1.0,43.262245,-79.920286,1280 Main Street W FL 2,L8S 4K1,TGXKszhNNure3bHQCbbTJA,restaurants


Time to fill in missing data.
- `restaurants` on Yelp -- "13065" on Foursquare
- `museums` on Yelp -- "10027" on Foursquare
- `amusementparks` on Yelp -- "10001" on Foursquare

In [388]:
pois_only.head()

,fsq_id,poi_name,f_category_id,poi_rating,poi_review_count,poi_latitude,poi_longitude,poi_address,postal_code,yelp_id,y_category_id
0,5dd2e5df0cd86e00081bd28b,1101 Convenience,13065.0,NaN,NaN,43.248628,-79.821565,1101 Cannon St E (Kensington),L8L 2J5,NaN,NaN
1,4dbb3d3ef7b1ab37dd3fe8f9,2 For 1 Pizza and Wings,13065.0,NaN,NaN,43.245058,-79.807569,158 Kenilworth Ave N,L8H 4R8,NaN,NaN
2,4f5e95a3e4b0253ade1e7454,241 Pizza,13065.0,NaN,NaN,43.250811,-79.851416,447 Main St E,L8N 1K1,NaN,NaN
3,017dcad7abe249b50f10d5ed,31 Service Battalion Museum,10027.0,NaN,NaN,43.275168,-79.856177,650 Catharine St N,NaN,NaN,NaN
4,59907f940868a228a8f2c2f2,337 Sketch Gallery,10027.0,NaN,NaN,43.251192,-79.816572,327 Ottawa St N (Barton St E),L8H 3Z8,NaN,NaN


In [389]:
# Fill missing f_category_id values
pois_only.loc[pois_only['f_category_id'] == 13065.0, 'y_category_id'] = 'restaurants' # Restaurants
pois_only.loc[pois_only['f_category_id'] == 10027.0, 'y_category_id'] = 'museums' # Museums
pois_only.loc[pois_only['f_category_id'] == 10001.0, 'y_category_id'] = 'amusementparks' # Amusement Parks

In [390]:
# Fill missing y_category_id values
pois_only.loc[pois_only['y_category_id'] == 'restaurants', 'f_category_id'] = 10027.0 # Restaurants
pois_only.loc[pois_only['y_category_id'] == 'museums', 'f_category_id'] = 10027.0 # Museums
pois_only.loc[pois_only['y_category_id'] == 'amusementparks', 'f_category_id'] = 10001.0 # Amusement Parks

In [391]:
pois_only.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1194 entries, 0 to 527
Data columns (total 11 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   fsq_id            666 non-null    object 
 1   poi_name          1194 non-null   object 
 2   f_category_id     1194 non-null   float64
 3   poi_rating        528 non-null    float64
 4   poi_review_count  528 non-null    float64
 5   poi_latitude      1194 non-null   float64
 6   poi_longitude     1194 non-null   float64
 7   poi_address       1192 non-null   object 
 8   postal_code       1163 non-null   object 
 9   yelp_id           528 non-null    object 
 10  y_category_id     1194 non-null   object 
dtypes: float64(5), object(6)
memory usage: 111.9+ KB


In [392]:
# Make our own ID
import random
import string

def generate_id():
    # Generate 2 random uppercase letters
    letters = ''.join(random.choices(string.ascii_uppercase, k=2))
    # Generate 4 random digits
    numbers = ''.join(random.choices(string.digits, k=4))
    # Combine letters and numbers
    return letters + numbers

pois_only['poi_id'] = pois_only['poi_name'].apply(lambda x: generate_id())

In [393]:
pois_only.columns

Index(['fsq_id', 'poi_name', 'f_category_id', 'poi_rating', 'poi_review_count',
       'poi_latitude', 'poi_longitude', 'poi_address', 'postal_code',
       'yelp_id', 'y_category_id', 'poi_id'],
      dtype='object')

In [394]:
# Standardize the columns
pois_only['poi_category'] = pois_only['y_category_id']

pois_only.columns

Index(['fsq_id', 'poi_name', 'f_category_id', 'poi_rating', 'poi_review_count',
       'poi_latitude', 'poi_longitude', 'poi_address', 'postal_code',
       'yelp_id', 'y_category_id', 'poi_id', 'poi_category'],
      dtype='object')

In [395]:
# Drop the old columns
pois_only.drop(columns=['f_category_id', 'y_category_id'], inplace=True)

In [396]:
pois_only.columns

Index(['fsq_id', 'poi_name', 'poi_rating', 'poi_review_count', 'poi_latitude',
       'poi_longitude', 'poi_address', 'postal_code', 'yelp_id', 'poi_id',
       'poi_category'],
      dtype='object')

In [397]:
pois_only.head()

,fsq_id,poi_name,poi_rating,poi_review_count,poi_latitude,poi_longitude,poi_address,postal_code,yelp_id,poi_id,poi_category
0,5dd2e5df0cd86e00081bd28b,1101 Convenience,NaN,NaN,43.248628,-79.821565,1101 Cannon St E (Kensington),L8L 2J5,NaN,RP1388,restaurants
1,4dbb3d3ef7b1ab37dd3fe8f9,2 For 1 Pizza and Wings,NaN,NaN,43.245058,-79.807569,158 Kenilworth Ave N,L8H 4R8,NaN,SY4399,restaurants
2,4f5e95a3e4b0253ade1e7454,241 Pizza,NaN,NaN,43.250811,-79.851416,447 Main St E,L8N 1K1,NaN,VX5752,restaurants
3,017dcad7abe249b50f10d5ed,31 Service Battalion Museum,NaN,NaN,43.275168,-79.856177,650 Catharine St N,NaN,NaN,BE5356,museums
4,59907f940868a228a8f2c2f2,337 Sketch Gallery,NaN,NaN,43.251192,-79.816572,327 Ottawa St N (Barton St E),L8H 3Z8,NaN,AW7335,museums


In [398]:
pois_only.nunique()

fsq_id               666
poi_name             932
poi_rating            41
poi_review_count      74
poi_latitude        1083
poi_longitude       1103
poi_address         1047
postal_code          465
yelp_id              528
poi_id              1194
poi_category           3
dtype: int64

In [399]:
# Back it up -- It's SQL ready.
pois_only.to_csv('./sql_ready/hamilton_pois.csv', index=False)

### Cleaning Combined Data

From `joining_data.ipynb`, I produced a dataset with all the POIs I was able to gather from both sources including the bike station info.

In [400]:
# Load the data
combined = pd.read_csv('../data/raw/combined_data.csv')

print(combined.shape)
print(combined.columns)
print(combined.nunique())
print(combined.info())

(19100, 19)
Index(['fsq_id', 'poi_name', 'f_category_id', 'poi_rating', 'poi_review_count',
       'poi_latitude', 'poi_longitude', 'poi_address', 'postal_code',
       'station_id', 'distance_from_station', 'station_name',
       'station_address', 'station_latitude', 'station_longitude',
       'available_bikes', 'empty_bike_slots', 'yelp_id', 'y_category_id'],
      dtype='object')
fsq_id                     669
poi_name                   932
f_category_id                3
poi_rating                  41
poi_review_count            74
poi_latitude              1086
poi_longitude             1106
poi_address               1047
postal_code                465
station_id                 190
distance_from_station    10509
station_name               190
station_address            163
station_latitude           190
station_longitude          190
available_bikes             19
empty_bike_slots            25
yelp_id                    529
y_category_id                3
dtype: int64
<class 'pa

In [401]:
combined.head()

,fsq_id,poi_name,f_category_id,poi_rating,poi_review_count,poi_latitude,poi_longitude,poi_address,postal_code,station_id,distance_from_station,station_name,station_address,station_latitude,station_longitude,available_bikes,empty_bike_slots,yelp_id,y_category_id
0,5ba0f82a15173e002c280c07,Synonym,13065.0,NaN,NaN,43.264558,-79.865376,328 James St N (Barton Street West),L8L 1H2,024a3edf037cb411d16acc08a7fcb954,420.0,Bay at Strachan,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,4fe7448be4b0faa0b99ed2c7,The Green Smoothie Bar,13065.0,NaN,NaN,43.262531,-79.866285,236 James St N,L8R 2L3,024a3edf037cb411d16acc08a7fcb954,606.0,Bay at Strachan,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,54cbb92b498ec28817437edb,Saint James Espresso Bar & Eatery,13065.0,NaN,NaN,43.260862,-79.866884,170 James St N (Cannon),L8R 2L1,024a3edf037cb411d16acc08a7fcb954,781.0,Bay at Strachan,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4b689c20f964a520be822be3,The Harbour Diner,13065.0,NaN,NaN,43.269474,-79.863286,486 James St N (btwn Ferrie St and Picton St),L8L 1J1,024a3edf037cb411d16acc08a7fcb954,415.0,Bay at Strachan,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,520e97ce498e3ee7817abaf4,Charred,13065.0,NaN,NaN,43.262759,-79.866231,244 James St N (btwn Robert St & Colbourne St),L8R 2L3,024a3edf037cb411d16acc08a7fcb954,583.0,Bay at Strachan,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [402]:
combined['station_address'] = citybikes['station_address']
combined['station_latitude'] = citybikes['station_latitude']
combined['station_longitude'] = citybikes['station_longitude']
combined['available_bikes'] = citybikes['available_bikes']
combined['empty_bike_slots'] = citybikes['empty_bike_slots']

combined.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19100 entries, 0 to 19099
Data columns (total 19 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   fsq_id                 8698 non-null   object 
 1   poi_name               18720 non-null  object 
 2   f_category_id          8698 non-null   float64
 3   poi_rating             10022 non-null  float64
 4   poi_review_count       10022 non-null  float64
 5   poi_latitude           18720 non-null  float64
 6   poi_longitude          18720 non-null  float64
 7   poi_address            18698 non-null  object 
 8   postal_code            18335 non-null  object 
 9   station_id             19100 non-null  object 
 10  distance_from_station  18720 non-null  float64
 11  station_name           19100 non-null  object 
 12  station_address        190 non-null    object 
 13  station_latitude       190 non-null    float64
 14  station_longitude      190 non-null    float64
 15  av

In [403]:
combined.head()

,fsq_id,poi_name,f_category_id,poi_rating,poi_review_count,poi_latitude,poi_longitude,poi_address,postal_code,station_id,distance_from_station,station_name,station_address,station_latitude,station_longitude,available_bikes,empty_bike_slots,yelp_id,y_category_id
0,5ba0f82a15173e002c280c07,Synonym,13065.0,NaN,NaN,43.264558,-79.865376,328 James St N (Barton Street West),L8L 1H2,024a3edf037cb411d16acc08a7fcb954,420.0,Bay at Strachan,"325 Bay Street North, Hamilton",43.267859,-79.867923,1.0,23.0,NaN,NaN
1,4fe7448be4b0faa0b99ed2c7,The Green Smoothie Bar,13065.0,NaN,NaN,43.262531,-79.866285,236 James St N,L8R 2L3,024a3edf037cb411d16acc08a7fcb954,606.0,Bay at Strachan,"2000 College Court, McMaster University, Hamilton",43.259178,-79.920503,2.0,12.0,NaN,NaN
2,54cbb92b498ec28817437edb,Saint James Espresso Bar & Eatery,13065.0,NaN,NaN,43.260862,-79.866884,170 James St N (Cannon),L8R 2L1,024a3edf037cb411d16acc08a7fcb954,781.0,Bay at Strachan,"62-64 Augusta Street, Hamilton",43.251632,-79.868980,0.0,10.0,NaN,NaN
3,4b689c20f964a520be822be3,The Harbour Diner,13065.0,NaN,NaN,43.269474,-79.863286,486 James St N (btwn Ferrie St and Picton St),L8L 1J1,024a3edf037cb411d16acc08a7fcb954,415.0,Bay at Strachan,"36 Hunter Street East, Hamilton",43.253330,-79.869533,6.0,11.0,NaN,NaN
4,520e97ce498e3ee7817abaf4,Charred,13065.0,NaN,NaN,43.262759,-79.866231,244 James St N (btwn Robert St & Colbourne St),L8R 2L3,024a3edf037cb411d16acc08a7fcb954,583.0,Bay at Strachan,"394 Maple Avenue, Hamilton",43.242163,-79.825003,4.0,5.0,NaN,NaN


In [404]:
# Fill missing f_category_id values
combined.loc[combined['f_category_id'] == 13065.0, 'y_category_id'] = 'restaurants' # Restaurants
combined.loc[combined['f_category_id'] == 10027.0, 'y_category_id'] = 'museums' # Museums
combined.loc[combined['f_category_id'] == 10001.0, 'y_category_id'] = 'amusementparks' # Amusement Parks

In [405]:
# Fill missing y_category_id values
combined.loc[combined['y_category_id'] == 'restaurants', 'f_category_id'] = 10027.0 # Restaurants
combined.loc[combined['y_category_id'] == 'museums', 'f_category_id'] = 10027.0 # Museums
combined.loc[combined['y_category_id'] == 'amusementparks', 'f_category_id'] = 10001.0 # Amusement Parks

In [406]:
combined.head()

,fsq_id,poi_name,f_category_id,poi_rating,poi_review_count,poi_latitude,poi_longitude,poi_address,postal_code,station_id,distance_from_station,station_name,station_address,station_latitude,station_longitude,available_bikes,empty_bike_slots,yelp_id,y_category_id
0,5ba0f82a15173e002c280c07,Synonym,10027.0,NaN,NaN,43.264558,-79.865376,328 James St N (Barton Street West),L8L 1H2,024a3edf037cb411d16acc08a7fcb954,420.0,Bay at Strachan,"325 Bay Street North, Hamilton",43.267859,-79.867923,1.0,23.0,NaN,restaurants
1,4fe7448be4b0faa0b99ed2c7,The Green Smoothie Bar,10027.0,NaN,NaN,43.262531,-79.866285,236 James St N,L8R 2L3,024a3edf037cb411d16acc08a7fcb954,606.0,Bay at Strachan,"2000 College Court, McMaster University, Hamilton",43.259178,-79.920503,2.0,12.0,NaN,restaurants
2,54cbb92b498ec28817437edb,Saint James Espresso Bar & Eatery,10027.0,NaN,NaN,43.260862,-79.866884,170 James St N (Cannon),L8R 2L1,024a3edf037cb411d16acc08a7fcb954,781.0,Bay at Strachan,"62-64 Augusta Street, Hamilton",43.251632,-79.868980,0.0,10.0,NaN,restaurants
3,4b689c20f964a520be822be3,The Harbour Diner,10027.0,NaN,NaN,43.269474,-79.863286,486 James St N (btwn Ferrie St and Picton St),L8L 1J1,024a3edf037cb411d16acc08a7fcb954,415.0,Bay at Strachan,"36 Hunter Street East, Hamilton",43.253330,-79.869533,6.0,11.0,NaN,restaurants
4,520e97ce498e3ee7817abaf4,Charred,10027.0,NaN,NaN,43.262759,-79.866231,244 James St N (btwn Robert St & Colbourne St),L8R 2L3,024a3edf037cb411d16acc08a7fcb954,583.0,Bay at Strachan,"394 Maple Avenue, Hamilton",43.242163,-79.825003,4.0,5.0,NaN,restaurants


In [407]:
# Standardize the columns
combined['poi_category'] = combined['y_category_id']
combined.drop(columns=['f_category_id', 'y_category_id'], inplace=True)

combined.head(2)

,fsq_id,poi_name,poi_rating,poi_review_count,poi_latitude,poi_longitude,poi_address,postal_code,station_id,distance_from_station,station_name,station_address,station_latitude,station_longitude,available_bikes,empty_bike_slots,yelp_id,poi_category
0,5ba0f82a15173e002c280c07,Synonym,NaN,NaN,43.264558,-79.865376,328 James St N (Barton Street West),L8L 1H2,024a3edf037cb411d16acc08a7fcb954,420.0,Bay at Strachan,"325 Bay Street North, Hamilton",43.267859,-79.867923,1.0,23.0,NaN,restaurants
1,4fe7448be4b0faa0b99ed2c7,The Green Smoothie Bar,NaN,NaN,43.262531,-79.866285,236 James St N,L8R 2L3,024a3edf037cb411d16acc08a7fcb954,606.0,Bay at Strachan,"2000 College Court, McMaster University, Hamilton",43.259178,-79.920503,2.0,12.0,NaN,restaurants


In [408]:
combined.shape

(19100, 18)

In [409]:
# Add poi_id
combined = pd.merge(pois_only, combined, how='left')

In [410]:
combined.head(2)

,fsq_id,poi_name,poi_rating,poi_review_count,poi_latitude,poi_longitude,poi_address,postal_code,yelp_id,poi_id,poi_category,station_id,distance_from_station,station_name,station_address,station_latitude,station_longitude,available_bikes,empty_bike_slots
0,5dd2e5df0cd86e00081bd28b,1101 Convenience,NaN,NaN,43.248628,-79.821565,1101 Cannon St E (Kensington),L8L 2J5,NaN,RP1388,restaurants,05fffdca01b4d886a823b025b6a04080,770.0,Maple at Rothsay - ERI08,NaN,NaN,NaN,NaN,NaN
1,5dd2e5df0cd86e00081bd28b,1101 Convenience,NaN,NaN,43.248628,-79.821565,1101 Cannon St E (Kensington),L8L 2J5,NaN,RP1388,restaurants,1846562b60f9d9469c055fed1d84898e,548.0,King George,NaN,NaN,NaN,NaN,NaN


In [411]:
combined.shape

(18675, 19)

In [412]:
# Complete as it can be...
combined.to_csv('./sql_ready/hamilton_city_combined.csv', index=False)

### Adding POI IDs to Foursquare and Yelp Data.

More cleaning and standardizing columns for SQL

In [413]:
# Load the data
pois_foursquare = pd.read_csv('./pois_only/pois_foursquare.csv')
pois_yelp = pd.read_csv('./pois_only/pois_yelp.csv')

print(pois_foursquare.shape)
print(pois_yelp.shape)

(666, 9)
(528, 9)


In [416]:
# Merge
f = pd.merge(pois_foursquare, pois_only, how='left')
y = pd.merge(pois_yelp, pois_only, how='left')

print(f"{f.shape} -- Foursquare")
print(f.columns)

print(f"{y.shape} -- Yelp")
print(y.columns)

(666, 12) -- Foursquare
Index(['fsq_id', 'poi_name', 'poi_category_id', 'poi_rating',
       'poi_review_count', 'poi_latitude', 'poi_longitude', 'poi_address',
       'postal_code', 'yelp_id', 'poi_id', 'poi_category'],
      dtype='object')
(528, 12) -- Yelp
Index(['yelp_id', 'poi_name', 'poi_category_id', 'poi_rating',
       'poi_review_count', 'poi_latitude', 'poi_longitude', 'poi_address',
       'postal_code', 'fsq_id', 'poi_id', 'poi_category'],
      dtype='object')


In [417]:
# Save to CSV
f.to_csv('./sql_ready/foursquare_data.csv', index=False)
y.to_csv('./sql_ready/yelp_data.csv', index=False)